# Numerical Stability Demo (QR vs SVD vs Normal Equations)

Synthetic demonstration comparing least-squares solvers on well- and ill-conditioned systems.

In [ ]:
import sys, os, time
import numpy as np
import matplotlib.pyplot as plt

# Ensure 'src' is importable
repo_root = os.path.abspath(os.path.join(os.getcwd()))
if os.path.isdir(os.path.join(repo_root, 'src')) and repo_root not in sys.path:
    sys.path.insert(0, repo_root)

from src.pcs_math.qr_householder import solve_via_qr
from src.pcs_math.svd_solve import svd_solve
from src.pcs_math.kahan import kahan_sum
np.set_printoptions(precision=4, suppress=True)
np.random.seed(0)

## LSQ on a well-conditioned system

In [ ]:
m, n = 100, 20
A = np.random.randn(m, n)
x_true = np.random.randn(n)
b = A @ x_true + 1e-10 * np.random.randn(m)

x_qr = solve_via_qr(A, b)
x_svd, info = svd_solve(A, b, return_info=True)
x_np = np.linalg.lstsq(A, b, rcond=None)[0]

print('well-conditioned residuals:')
for name, x in [('qr', x_qr), ('svd', x_svd), ('np', x_np)]:
    r = np.linalg.norm(A @ x - b)
    print(f'  {name}: {r:.2e}')

## LSQ on an ill-conditioned system

In [ ]:
U = np.random.randn(n, n)
V = np.random.randn(n, n)
s = np.logspace(0, -12, n)
A_ill = U @ np.diag(s) @ V.T
x_true2 = np.random.randn(n)
b2 = A_ill @ x_true2

# Normal equations (unstable)
AtA = A_ill.T @ A_ill
Atb = A_ill.T @ b2
try:
    x_ne = np.linalg.solve(AtA, Atb)
except np.linalg.LinAlgError:
    x_ne = np.linalg.lstsq(AtA, Atb, rcond=None)[0]

x_qr2 = solve_via_qr(A_ill, b2)
x_svd2, info2 = svd_solve(A_ill, b2, rcond=1e-10, return_info=True)

errs = {
    'normal_eq': np.linalg.norm(x_ne - x_true2),
    'qr': np.linalg.norm(x_qr2 - x_true2),
    'svd': np.linalg.norm(x_svd2 - x_true2),
}
errs

## Kahan summation vs naive
Large + many tiny numbers.

In [ ]:
large = 1.0
small = 1e-10
N = 100000
x = np.array([large] + [small]*N)
expected = large + N*small
naive = float(sum(x))
numpy_sum = float(np.sum(x))
kahan = float(kahan_sum(x))
print('abs error naive   ', abs(naive-expected))
print('abs error numpy   ', abs(numpy_sum-expected))
print('abs error kahan   ', abs(kahan-expected))